<a href="https://colab.research.google.com/github/noambassat/AI_Agent_Nayara/blob/main/ai_agent_nayara.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U langchain-openai --quiet

In [ ]:
# Imports
from langchain.memory import ConversationBufferMemory
from openai import OpenAI
import os
from google.colab import userdata

In [ ]:
# Initialize OpenAI GPT-4o Client
client = OpenAI(api_key=userdata.get('open_ai_key'))

In [ ]:
# GPT prompt sender function
def ask_gpt(prompt):
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7
        )
        return response.choices[0].message.content.strip().strip('"')
    except Exception as e:
        print(f"Error: {e}")
        return ""

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

def get_chat_history():
  return memory.load_memory_variables({})["chat_history"]

# Part 1 -  Fixed Questions

In [ ]:
questions = {
  "first_name": "What is your first name?",
  "age": "Can you please share your age?",
  "diagnosis": "What type of breast cancer diagnosis did you receive?",
  "treatment": "What treatment are you currently receiving, if any?",
  "support_system": "Do you have a support system like family or friends?"
}

user_profile = {}

In [ ]:
# Evaluates a user's answer to a specific question using GPT.
# Returns either a clear extracted answer or an empathetic follow-up prompt.

def judge_layer(question, answer, chat_history):
    prompt = f"""
      You are a judgment module for an empathetic AI supporting users recently diagnosed with breast cancer.

      CONTEXT:
      Conversation history:
      {chat_history}

      QUESTION:
      "{question}"

      USER'S RESPONSE:
      "{answer}"

      TASK:
      Evaluate the user's response and determine whether it clearly answers the question.

      - If it does, respond:
        CLEAR|[Extracted concise answer]

      - If it does NOT clearly answer the question, respond:
        FOLLOWUP|[Empathetic, focused message for the AI agent to say next. This message should include an adapted restatement of the original question.]

      RULES:
      - Be emotionally sensitive and vary phrasing to avoid repetition.
      - When generating FOLLOWUP responses, always vary the wording from previous attempts. Do not repeat the same phrasing or sentence structure from earlier messages, even if the user's response remains similar.
      - NEVER ask broad or generic questions like "Anything else?" or "How can I help you?"
      - ALWAYS stay focused on the original question.
      - Evaluate the latest user response in light of the full conversation history.

      Answer Extraction Notes:
      - If the user's response clearly answers the question but includes emotional, narrative, or informal language, extract a concise, structured version that preserves the core meaning.
      - Summarize emotional or nuanced responses into a brief, useful format suitable for a user profile (e.g., "No steady support system" instead of "people come and go").

      Special Cases:
      1. If the user explicitly refuses to answer - acknowledge gently, explain why it helps, and suggest revisiting later.
      2. If the user is vague or hesitant - encourage gently and restate the question clearly.
      3. If the user asks a relevant question (e.g., about types of cancer) -
        briefly answer their question with helpful, accurate information,
        then immediately return to the original question in the same message.
      4. If the user asks something unrelated - gently acknowledge and return to the original question, without continuing the off-topic thread.

      RESPONSE FORMAT:
      Either:
      CLEAR|[answer]
      or:
      FOLLOWUP|[response message]

      No explanations. Respond in the exact format.
      """

    return ask_gpt(prompt)

In [ ]:
# Asks the user a question up to max_attempts times.
# Uses the judge layer to evaluate clarity.
# Saves clear answers to user_profile. Otherwise - asks follow Ups.
# Prints supportive messages if no clear answer is received.

def handle_question(key, question, user_profile, max_attempts=3, essential=False):
  attempts = 0
  current_question = question

  while attempts < max_attempts:
      user_response = input(f"{current_question} ").strip()
      memory.save_context({"input": current_question}, {"output": user_response})
      chat_history = get_chat_history()

      judge_response = judge_layer(question, user_response, chat_history)

      if judge_response.startswith("CLEAR|"):
          extracted_answer = judge_response.split("CLEAR|")[1].strip()
          user_profile[key] = extracted_answer
          print("Thank you for sharing.")
          return True
      elif judge_response.startswith("FOLLOWUP|"):
          current_question = judge_response.split("FOLLOWUP|")[1].strip()
          attempts += 1

  if max_attempts != 1:
    msg = "We'll revisit shortly." if essential else "Let's revisit this later."
    print(f"I understand this might be difficult right now. {msg}")
  return False

In [ ]:
# Runs the full question flow:
# Phase 1 - Asks each question once (with internal retries).
# Phase 2 - Repeats only unanswered questions until answered or skipped.


def run_interaction_flow(user_profile=None):
  if user_profile is None: user_profile = {}

  # Phase 1 – Initial attempts
  for key in questions: handle_question(key, questions[key], user_profile)

  # Phase 2 – Empty responses
  for key in questions:
      while key not in user_profile or not user_profile[key].strip():
          print(f"\nLet's revisit this important question: {questions[key]}")
          handle_question(key, questions[key], user_profile, essential=True)

  print("\nFinal User Profile Collected:")
  for k in questions:
      print(f"{k.capitalize()}: {user_profile.get(k, 'Not fully answered')}")


In [ ]:
run_interaction_flow()

What is your first name? why do you need it?
I understand if you're feeling cautious about sharing your first name. It helps us personalize our support for you. Could you let me know your first name when you feel comfortable? no..
I understand your hesitation, and it's completely okay not to share right now. If you change your mind later, please feel free to let us know your first name so we can make your experience more personalized. I don't need it more personalized!
I understand this might be difficult right now. Let's revisit this later.
Can you please share your age? 23
Thank you for sharing.
What type of breast cancer diagnosis did you receive? I can't remember it right now
I understand it might be hard to recall details right now, and that's okay. Whenever you're ready, could you let us know the type of breast cancer diagnosis you received? It can help us provide more specific support. What are the most common breast cancer among young woman?
It sounds like you're curious about 

In [ ]:
run_interaction_flow()

What is your first name? N
I understand if you're not ready to share your full name right now, and that's perfectly okay. If you feel comfortable, would you consider sharing your first name with me when you're ready? It can help make our conversation more personalized. Nava
Thank you for sharing.
Can you please share your age? 55
Thank you for sharing.
What type of breast cancer diagnosis did you receive? TNBC
Thank you for sharing.
What treatment are you currently receiving, if any? Nutrition based therapy
Thank you for sharing.
Do you have a support system like family or friends? what do you mean?
A support system often includes family, friends, or other individuals who can provide emotional and practical help during challenging times. Do you have people in your life who you feel supported by? It's not that simple.. People come and go and scared to deal with that...
It sounds like having a steady support system is complex for you. It's important to have people who can provide emotion

# Part 2 - Dynamic Dialogue Generation

In [ ]:
import json
import re
from langchain.schema import AIMessage, HumanMessage

In [ ]:
# Prompts the user to describe their medical condition in free text.
# Extracts structured details using GPT and returns them as a JSON object.
# Saves the result for later use.


def generate_condition_profile(user_input=None):
  if user_input is None: user_input = input("Please briefly describe your condition and treatment plan: ")

  condition_profile_prompt = f"""
        You will receive a short free-form medical description from a user recently diagnosed with breast cancer.

        Your task is to extract any relevant structured details mentioned in the text, and output them as a clean, flat JSON object.

        Do not assume a fixed schema. The field names should reflect only the content actually provided by the user.

        Guidelines:
        - Use simple, descriptive field names (e.g., "age", "treatment_type", "biopsy_result")
        - Only include fields explicitly mentioned
        - If the information is partial or informal, preserve it as-is
        - Do not invent or infer missing details

        Example:

        User input:
        "I'm Shila, age 48, just had a biopsy, and they're still checking if it's invasive. MRI coming up."

        Output:
        {{
          "name": "Shila",
          "age": 48,
          "biopsy_status": "still checking if it's invasive",
          "upcoming_procedure": "MRI"
        }}

        Now process the following user input:
        "{user_input}"

        Return only the JSON.
        """


  condition_profile_json = ask_gpt(condition_profile_prompt)

  condition_profile_json_clean = condition_profile_json.strip("```json").strip("```").strip()

  condition_profile = json.loads(condition_profile_json_clean)

  memory.save_context(
  {"condition_profile": json.dumps(condition_profile)},
  {"output": "Condition profile stored successfully."}
  )
  return condition_profile

In [ ]:
condition_profile = generate_condition_profile()
# Sarah, age 45, diagnosed with Triple-Negative breast cancer, at IIb stage. Just started the AC-T chemotherapy. Just schedules the Genetic testing and breast MRI for next week.
condition_profile

Please briefly describe your condition and treatment plan: Sarah, age 45, diagnosed with Triple-Negative breast cancer, at IIb stage. Just started the AC-T chemotherapy. Just schedules the Genetic testing and breast MRI for next week.


{'name': 'Sarah',
 'age': 45,
 'diagnosis': 'Triple-Negative breast cancer',
 'cancer_stage': 'IIb',
 'current_treatment': 'AC-T chemotherapy',
 'upcoming_tests': 'Genetic testing and breast MRI for next week'}

In [ ]:
# Generates one personalized, empathetic follow-up question based on the user's condition and conversation history.
# Avoids repetition and respects previously declined topics.

def generate_dynamic_question(condition_profile, chat_history):
    formatted_history = json.dumps(chat_history, ensure_ascii=False, indent=2)

    prompt = f"""
        You are an empathetic conversational agent supporting a woman recently diagnosed with breast cancer.

        Use the following context:

        Medical profile:
        {condition_profile}

        Conversation history:
        {formatted_history}

        Your task:
        Write one emotionally sensitive and personalized follow-up question that:
        - Clearly relates to her current condition, treatment, upcoming procedures, or emotional state.
        - Gently invites reflection or emotional sharing, based on her previous answers.
        - Does NOT revisit topics she explicitly refused to discuss.
        - Do not ask about the same topic more than once, even if phrased differently. Always scan the full conversation history to avoid redundancy.
        - If a topic has already been addressed, choose a new, relevant direction based on her needs.
        - Vary tone, vocabulary, and sentence structure to maintain a natural, non-repetitive flow.
        - End with one single, clearly focused question.
        - Do not include more than one question in a sentence.

        Output format:
        Only return the question as plain text — no extra text, no formatting, and no quotation marks.
        """

    return ask_gpt(prompt)


In [ ]:
# Runs a dynamic interaction loop to ask 5 personalized questions.
# Each question is generated based on the user's medical profile and chat history.
# Saves each question to memory and updates the condition profile with responses.

def dynamic_interaction_flow(condition_profile):
  for question_number in range(5):
      # Load the raw memory objects
      raw_history = memory.load_memory_variables({})["chat_history"]

      # Convert to role-based message format
      chat_history = []
      for msg in raw_history:
          if isinstance(msg, HumanMessage):
              chat_history.append({"role": "user", "content": msg.content})
          elif isinstance(msg, AIMessage):
              chat_history.append({"role": "assistant", "content": msg.content})

      next_question = generate_dynamic_question(condition_profile, chat_history)

      handle_question(f"dynamic_question_{question_number}", next_question, condition_profile, max_attempts=1)

      memory.save_context({"input": next_question}, {"output": "Question generated successfully."})

  print("\nFinal Dynamic Interaction Complete.")

  print("\nFinal Profile Collected:")
  for key, value in condition_profile.items():
      print(f"{key.capitalize()}: {value}")


In [ ]:
dynamic_interaction_flow(generate_condition_profile())

Please briefly describe your condition and treatment plan: Sarah, age 45, diagnosed with Triple-Negative breast cancer, at IIb stage. Just started the AC-T chemotherapy. Just schedules the Genetic testing and breast MRI for next week.
How are you feeling about starting your AC-T chemotherapy and the upcoming genetic testing and breast MRI? Terrified 
Thank you for sharing.
How do you feel about the support system you have around you as you navigate this journey? I don't have
As you prepare for the upcoming genetic testing and breast MRI, is there anything specific you’re hoping to learn or understand better about your diagnosis? No
Thank you for sharing.
What are some ways you find comfort or strength when you're feeling overwhelmed during this journey? Sport helps me
Thank you for sharing.
How do you feel sport has helped you emotionally during this challenging time? It's clearing my mind
Thank you for sharing.

Final Dynamic Interaction Complete.

Final Profile Collected:
Name: Sarah